In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import photutils
import datetime
from astropy.visualization import astropy_mpl_style
plt.style.use(astropy_mpl_style)


import astropy
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
from photutils.detection import DAOStarFinder
from photutils.aperture import CircularAperture, CircularAnnulus, ApertureStats
from astropy.stats import sigma_clipped_stats
from astropy.table import Column
import glob
import time

In [ ]:
filepath = '/Users/maxrosenblatt/Desktop/AstroData/2023_1_23/Light/Blue/Reduced/'

maxstarsums = []
filetimes = []

flist = sorted(glob.glob(filepath+"Reduced_star_Light_*.fits"))


for file in flist:
    image_file = get_pkg_data_filename(file)
    image_data = fits.getdata(image_file, ext=0)
    fitsFile = fits.open(file)
    date_obs = fitsFile[0].header['DATE-OBS']
    filename = (file.replace("/Volumes/maxdrive/AstroData/2023_1_23/Light/Blue/Reduced/",""))
    time = date_obs[11:19]
    date = date_obs[:10]
    data = fitsFile[0].data
    median = np.median(data)
    tol = 5*median
    fwhm = 10
    
    dao = DAOStarFinder(tol, fwhm)
    stars = dao(data)

    positions = []

    for i in range(len(stars)):
        posn = (stars['xcentroid'][i], stars['ycentroid'][i])
        positions.append(posn)
        
    apertures = CircularAperture(positions, r=3.5*fwhm)
    annulus_apertures = CircularAnnulus(positions, r_in=3.5*fwhm, r_out=(3.5*fwhm*np.sqrt(5)))
    aperstats = photutils.aperture.ApertureStats(image_data, apertures)
    
    columns = ('id', 'mean', 'median', 'std', 'var', 'sum')
    stats_table = aperstats.to_table(columns)
    for col in stats_table.colnames:
        stats_table[col].info.format = '%.8g'  # for consistent table output
    
        
    idx = np.argmax(stats_table['sum'])
    mainstarstats = stats_table[idx]

    """
    if (fitsFile[0].header['EXPTIME'] == 45):
        mag = (-2.5*np.log10(mainstarstats['sum']/45))
        filetimes.append([filename, date+" "+time, (mainstarstats['sum']*(30/45))])
        print (str(filetimes[(len(filetimes)-1)])+" SCALED")
    else:
        mag = (-2.5*np.log10(mainstarstats['sum']/45))
        filetimes.append([filename, date+" "+time, (mainstarstats['sum'])])    
        print (filetimes[(len(filetimes)-1)])
    """
    
    mag = (-2.5*np.log10(mainstarstats['sum']/(fitsFile[0].header['EXPTIME'])))
    filetimes.append([filename, date+" "+time, mag])
    print (filetimes[(len(filetimes)-1)])
    

    


    
    

df = pd.DataFrame (filetimes, columns = ['file', 'datetime', 'mag'])
df['datetime'] = pd.to_datetime(df['datetime'])
df_sorted = df.sort_values('datetime')
df_sorted
df_sorted.to_csv('stars(mag).csv')

In [ ]:
fig = plt.figure(figsize = (15,7))
plt.gca().invert_yaxis()
plt.scatter(df_sorted['datetime'], df_sorted['mag'], marker = 'x', color = 'black')